In [20]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/insurance-data/insurance_data.csv


In [21]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline

In [22]:
df = pd.read_csv("/kaggle/input/insurance-data/insurance_data.csv")
df.head()

,age,bought_insurance
0,22,0
1,25,0
2,47,1
3,52,0
4,46,1


In [23]:
# Define sigmoid function
def sigmoid_numpy(X):
    return 1 / (1 + np.exp(-X))

In [24]:
# Define log loss function
def log_loss(y_true, y_predicted):
    epsilon = 1e-15
    y_predicted_new = np.clip(y_predicted, epsilon, 1 - epsilon)
    return -np.mean(y_true * np.log(y_predicted_new) + (1 - y_true) * np.log(1 - y_predicted_new))


In [36]:
# Define neural network class
class myNN:
    def __init__(self):
        self.w1 = 1  # Weight for the feature
        self.bias = 0  # Bias term

    def fit(self, X, y, epochs, loss_threshold):
        self.w1, self.bias = self.gradient_descent(X, y, epochs, loss_threshold)
        print(f"Final weight and bias: w1: {self.w1}, bias: {self.bias}")

    def predict(self, X_test):
        weighted_sum = self.w1 * X_test + self.bias
        return sigmoid_numpy(weighted_sum)

    def gradient_descent(self, age, y_true, epochs, loss_threshold):
        w1 = 1
        bias = 0
        rate = 0.5
        n = len(age)
        
        for i in range(epochs):
            weighted_sum = w1 * age + bias
            y_predicted = sigmoid_numpy(weighted_sum)
            loss = log_loss(y_true, y_predicted)
            
            # Compute gradients
            error = y_predicted - y_true
            w1d = np.mean(error * age)  # Gradient for weight
            bias_d = np.mean(error)  # Gradient for bias
            
            # Update weights and bias
            w1 -= rate * w1d
            bias -= rate * bias_d
            
            if i % 50 == 0:
                print(f'Epoch: {i}, w1: {w1}, bias: {bias}, loss: {loss}')
            
            if loss <= loss_threshold:
                print(f'Epoch: {i}, w1: {w1}, bias: {bias}, loss: {loss}')
                break

        return w1, bias


In [37]:
df.head()

,age,bought_insurance
0,22,0
1,25,0
2,47,1
3,52,0
4,46,1


In [42]:
# Preprocess data
X = df['age'].values
y = df['bought_insurance'].values

In [43]:
# Normalize features (optional, depending on the dataset)
X_scaled = (X - X.mean()) / X.std()  # Simple standardization

In [44]:
# Split dataset (example split, you might need to adjust according to your dataset)
from sklearn.model_selection import train_test_split
X_train_scaled, X_test_scaled, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


In [45]:
# Train and evaluate model
customModel = myNN()
customModel.fit(X_train_scaled, y_train, epochs=8000, loss_threshold=0.4631)


Epoch: 0, w1: 1.0533173105089206, bias: 0.03026740555547792, loss: 0.4849713878575394
Epoch: 4, w1: 1.223846263654187, bias: 0.12159248743244681, loss: 0.46309821362368747
Final weight and bias: w1: 1.223846263654187, bias: 0.12159248743244681


In [46]:
# Predict on test set
y_predicted = customModel.predict(X_test_scaled)

In [47]:
# Evaluate model
test_loss = log_loss(y_test, y_predicted)
print(f'Test Loss: {test_loss}')

Test Loss: 0.2712659016920223
